Notebook to investigate CNN - mass - flux

In [2]:
# imports
# import numpy as np
#!pip install pandas
# import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers as tfkl
import tensorflow_datasets as tfds
# import my_dataset
import matplotlib.pyplot as plt
data_dir='/gpfsscratch/rech/qrc/commun/'

In [ ]:
pwd

In [3]:
# functions

def Model(nb_summaries):
    """ Creates a small fully connected network
    """
    return tf.keras.Sequential([
        tfkl.Input(shape=(143)),
        tfkl.Dense(16, activation='relu'),
        tfkl.Dense(32, activation='relu'),
        tfkl.Dense(64, activation='relu'),
        tfkl.Dense(128, activation='relu'),
        tfkl.Dense(nb_summaries, activation='softplus')
        ])


# def preprocessing(example):
#     img = tf.math.asinh(example['image'] / tf.constant(scaling) / 3. )
#   # We return the image as our input and output for a generative model
#   return img

def input_fn(mode='train', batch_size=64):
    """
    mode: 'train' or 'test'
    """
    if mode == 'train':
        dataset = tfds.load('sfhsed', split='train[:80%]', data_dir=data_dir)
        dataset = dataset.repeat()
        dataset = dataset.shuffle(10000)
    else:
        dataset = tfds.load('sfhsed', split='train[80%:]')

    dataset = dataset.batch(batch_size, drop_remainder=True)
#     dataset = dataset.map(preprocessing) # Apply data preprocessing
    dataset = dataset.prefetch(-1)  # fetch next batches while training current one (-1 for autotune)
    return dataset

In [4]:
model = Model(1)
# print(model.summary)
# dset = input_fn()

# model.compile(optimizer=tf.keras.optimizers.Adam(),
#               loss=tf.keras.losses.MSE)

# history = model.fit(x=dset['flux'], y=dset['quantile'][4], epochs=20)

# plt.plot(history.loss)


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                2304      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 13,409
Trainable params: 13,409
Non-trainable params: 0
_________________________________________________________________


In [6]:
dset = input_fn()

In [7]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.MSE)

In [29]:
dset

<PrefetchDataset shapes: {flux: (64, 143), mass: (64, 100), object_id: (64,), quantile: (64, 9), time: (64, 100)}, types: {flux: tf.float32, mass: tf.float32, object_id: tf.float32, quantile: tf.float32, time: tf.float32}>

In [8]:
def train_step(x):
    with tf.GradientTape() as tape:
        predicted_quantile = model(x['flux'])
        loss = tf.keras.losses.MSE(x['quantile'][4], predicted_quantile)
        grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

In [9]:
from tqdm import trange

In [ ]:
loss_history = []
optimizer = tf.keras.optimizers.Adam()
nb_epochs = 10
t = trange(nb_epochs, desc='training')
for epoch in t:
    for i, features in enumerate(dset):
        loss = train_step(features)
        loss_history.append(loss)

training:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
history = model.fit(x=dset['flux'], y=dset['quantile'][4], epochs=10)